In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

In [2]:
# 定义Add & Norm层
class AddNorm(nn.Module):
    def __init__(self, hidden_dim):
        super(AddNorm, self).__init__()
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, x, residual):
        return self.norm(x + residual)

# 定义Feed Forward层
class FeedForward(nn.Module):
    def __init__(self, hidden_dim, ff_dim):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(hidden_dim, ff_dim)
        self.linear2 = nn.Linear(ff_dim, hidden_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

# 定义Multi-Head Attention层
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads

        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)

        self.fc = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        Q = self.query(query).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(key).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(value).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        energy = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_dim)

        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        attention = torch.softmax(energy, dim=-1)

        x = torch.matmul(attention, V).transpose(1, 2).contiguous().view(batch_size, -1, self.hidden_dim)
        x = self.fc(x)

        return x

# 定义Encoder层
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_heads, ff_dim):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(hidden_dim, num_heads)
        self.ff = FeedForward(hidden_dim, ff_dim)
        self.norm1 = AddNorm(hidden_dim)
        self.norm2 = AddNorm(hidden_dim)

    def forward(self, x, mask=None):
        residual = x
        x = self.self_attn(x, x, x, mask)
        x = self.norm1(x, residual)

        residual = x
        x = self.ff(x)
        x = self.norm2(x, residual)

        return x

# 定义Decoder层
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_heads, ff_dim):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(hidden_dim, num_heads)
        self.cross_attn = MultiHeadAttention(hidden_dim, num_heads)
        self.ff = FeedForward(hidden_dim, ff_dim)
        self.norm1 = AddNorm(hidden_dim)
        self.norm2 = AddNorm(hidden_dim)
        self.norm3 = AddNorm(hidden_dim)

    def forward(self, x, enc_outputs, src_mask=None, tgt_mask=None):
        residual = x
        x = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x, residual)

        residual = x
        x = self.cross_attn(x, enc_outputs, enc_outputs, src_mask)
        x = self.norm2(x, residual)

        residual = x
        x = self.ff(x)
        x = self.norm3(x, residual)

        return x
class PositionalEncoding(nn.Module):
    def __init__(self, hidden_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, hidden_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_dim, 2).float() * (-np.log(10000.0) / hidden_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x
class Transformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads, num_layers, ff_dim):
        super(Transformer, self).__init__()
        self.input_embedding = nn.Linear(input_dim, hidden_dim)
        self.output_embedding = nn.Linear(hidden_dim, output_dim)

        self.encoder_layers = nn.ModuleList([EncoderLayer(hidden_dim, num_heads, ff_dim) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(hidden_dim, num_heads, ff_dim) for _ in range(num_layers)])

    def forward(self, src, tgt_mask=None):
        src_embed = self.input_embedding(src)

        enc_outputs = src_embed
        for enc_layer in self.encoder_layers:
            enc_outputs = enc_layer(enc_outputs)

        dec_outputs = enc_outputs
        for dec_layer in self.decoder_layers:
            dec_outputs = dec_layer(dec_outputs, enc_outputs, tgt_mask=tgt_mask)

        outputs = self.output_embedding(dec_outputs[:, -1, :])
        return outputs

In [3]:

instrument_list=[
                '600519.SH',
                '002304.SZ',
                '000568.SZ',
                '600000.SH',
                '601988.SH',
                '601398.SH',
                '601288.SH',
                '600919.SH',
                '603259.SH',
                '002737.SZ',
                '600566.SH',
                '300015.SZ',
                '600721.SH',
                '300750.SZ',
                '002455.SZ',
                '600104.SH',
                '002594.SZ',
                '601633.SH'
                ]

#设置训练设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 设置模型参数
input_dim = 20
hidden_dim = 128
output_dim = 1
num_heads = 8
num_layers = 6
ff_dim = 512

# 创建模型实例
model = Transformer(input_dim, hidden_dim, output_dim, num_heads, num_layers, ff_dim)
model.to(device)

path="128.pt"
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [4]:
data = pd.read_csv('tmp2.csv')

features = ['instrument','company_id','open', 'close', 'high', 'low', 'volume', 'money_netflow', 'money_inflow', 'money_outflow',
        'net_inflow_rate', 'list_sector', 'CPI', '无风险利率',
        'total_market_cap', 'float_market_cap', 'pe_ttm', 'pb',
        'dividend_yield_ratio', 'major_id', 'minor_id',
        'change_ratio']
data = data[features]


def collect(data:pd.DataFrame,instrument_code:str)->np.ndarray:
            '''
            返回指定股票的价格和收益率
            '''
            minidata = data[data.instrument==instrument_code]
            tmp = minidata.values[:,1:]
            close_price = tmp[:,2] # close price
            ratio = tmp[:,-1] # ratio
            return close_price, ratio


def create_dataset(data, look_back=5):
        X, Y, Z = [], [], []
        for i in range(len(data) - look_back - 1):
            X.append(data[i:(i + look_back), :-1])
            Y.append(data[i + look_back, -1])
            #Z.append(data[i+1:(i + look_back),-1])
        return np.array(X), np.array(Y)#, np.array(Z)


def create_all_dataset(data, instrument_list, look_back=5):
        X, Y = [], []
        # 划分
        for i in instrument_list:
            minidata=data[data.instrument==i]
            x, y = create_dataset(minidata.values[:,1:], look_back)#去除instrument列 # array合并
            X.append(x)
            Y.append(y)
            #Z.append(z)
        return X, Y

    # 创建 StandardScaler 对象
scaler = StandardScaler()

    # 对数据进行归一化处理
scale_need_cols = [ 'open', 'close', 'high', 'low', 'volume', 'money_netflow', 'money_inflow', 'money_outflow',
        'net_inflow_rate', 'CPI', '无风险利率',
        'total_market_cap', 'float_market_cap', 'pe_ttm', 'pb',
        'dividend_yield_ratio',
        'change_ratio']

data_copy = data.copy()
data_copy.loc[:, scale_need_cols] = scaler.fit_transform(data_copy[scale_need_cols])

look_back = 5
X, Y = create_all_dataset(data_copy, instrument_list, look_back)

In [5]:
def trade_opt(lack, Capital, q_current, index, model, X, instrument_list, n=len(instrument_list)):
    '''
    n           资产数量
    Capital     投资资本 优化完成要重新计算buy,sell带来的变化
    p           当前资产价格向量
    p_prime     预测的资产价格向量
    R           预测收益率向量
    c = 1e-4    交易成本率
    alpha = 0.5 风险容忍度
    Sigma       资产收益率的协方差矩阵
    q_current   当前的资产持有量向量



    '''



    Capital_init=1e4

    def rearrange(X, n, index):
            tmp = {}
            for j in range(n):
                tmp[j] = X[j][index]   # (236,5,20)
            return tmp
    
    _ = rearrange(X, 18, index = index-1)           

    def inverse_transform_column(X, scaler, column_number):
        # 获取原始数据的均值和标准差
        mean = scaler.mean_[column_number]  # 最后一列的均值
        scale = scaler.scale_[column_number]  # 最后一列的标准差

        # 创建一个副本,避免修改原始数据
        X_inv = np.array(X).copy()

        # 对最后一列进行反归一化
        X_inv = X_inv * scale + mean

        return X_inv

    def predict(model,X:np.ndarray)->np.ndarray:
        model.eval()
        model.to('cpu')
        # 预测收益率向量
        X=X.astype(float)
        #.reshape(18,5,20)
        Input_tensor=torch.tensor(X,dtype=torch.float32)
        with torch.no_grad():
            R_pred = model(Input_tensor)
            R_pred_inverse = inverse_transform_column(R_pred, scaler, -1)
            R = R_pred_inverse.flatten()
        return R
    
    #n = 18 # 资产数量 也放入
    #index = 1 # window_slice_index 放入函数参数了

    p_previous = []
    R_previous = []
    p_real = []

    for i in instrument_list:
        p1,R1 = collect(data = data,instrument_code = i)
        p_previous.append(p1[index:index+4])
        R_previous.append(R1[index:index+4])
        p_real.append(p1[index+4])

    p_real = np.stack(p_real)
    p_previous = np.stack(p_previous)
    R_previous = np.stack(R_previous)

    input = np.stack(list(_.values()))

    R_p = predict(model, input) # X (18,5,20)-> R_p (18,)

    R_full = np.hstack((R_previous, R_p.reshape(18,1))) # (18,) + (18,4) -> (19,5)

    p_4 = []
    for i in range(n):
        p_4.append(p_previous[i][-1])
    p_4 = np.array(p_4)
    p_full = np.hstack((p_previous,((R_p+1)*p_4).reshape(18,1)))
    R_full = R_full.astype(float)


    p = p_full[:,-2] # 当前资产价格向量
    p_prime = p_full[:,-1] # 预测的资产价格向量
    R = R_p # 预测收益率向量
    c = 1e-4 # 交易成本率
    alpha = 1e-4 # 风险容忍度
    Sigma = np.cov(R_full)

    ''' Section Gurobi'''
    # 创建优化模型
    m = Model("Portfolio Optimization")
    m.Params.NumericFocus = 3
    if Capital<0:
         lack-=Capital
         Capital=0
    # 定义决策变量
    q = m.addVars(n, lb=0, vtype=GRB.INTEGER, name="q")
    #q[0].setAttr("vtype", GRB.CONTINUOUS) # 除第一个元素外,其他元素为整数
    w = m.addVars(n, lb=0, ub=1, name="w")
    buy = m.addVars(n, lb=0, name="buy")
    sell = m.addVars(n, lb=0, name="sell")

    # 设置目标函数
    obj = quicksum(q[i] * p_prime[i] for i in range(n)) - quicksum(c * (buy[i] + sell[i]) * p[i] for i in range(n))
    m.setObjective(obj, GRB.MAXIMIZE)

    # 添加约束条件
    m.addConstr(Capital >= quicksum(p[i] * q[i] for i in range(n)), "budget")
    m.addConstrs((quicksum(p[j] * q[j] for j in range(n))*w[i] == p[i] * q[i]   for i in range(n)), "weights")
    m.addConstr(
        quicksum(Sigma[i][j] * w[i] * w[j] for i in range(n) for j in range(n)) <= alpha, 
        "risk"
    )
    m.addConstrs((q[i] == q_current[i] + buy[i] - sell[i] for i in range(n)), "balance")

    # 求解模型
    m.optimize()

    print("Index:",index)
    sell=[var.X for var in sell.values()]
    print("Sell:",sell)
    buy=[var.X for var in buy.values()]
    print("Buy:",buy)
    q=[var.X for var in q.values()]
    print("Q:",q)
    cost = c*((buy @ p) + (sell @ p))
    Capital_remain = Capital -(buy @ p) + (sell @ p) - cost
    print("Capital_remain", Capital_remain)
    real_Profit = q @ p_real - Capital_init - cost + Capital_remain
    print("real_Profit:", real_Profit)
    print("lack:",lack)
    return lack, q, Capital_remain # q->q_current, Capital_remain->Capital
    

In [6]:
Capital_remain = 1e4
q = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
lack=0
for index in range(1,235):#235
        lack, q, Capital_remain = trade_opt(lack, Capital_remain, q, index, model, X, instrument_list)

Set parameter NumericFocus to value 3
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i7-13700K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 19 rows, 72 columns and 72 nonzeros
Model fingerprint: 0x446bdf6a
Model has 19 quadratic constraints
Variable types: 54 continuous, 18 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [2e-07, 1e+04]
  QLMatrix range   [5e+00, 1e+04]
  Objective range  [5e-04, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+04, 1e+04]
  QRHS range       [1e-04, 1e-04]
Found heuristic solution: objective -0.0000000
Presolve added 7 rows and 0 columns
Presolve removed 0 rows and 25 columns
Presolve time: 0.00s
Presolved: 1267 rows, 353 columns, 3110 nonzeros
Presolved model has 4 SOS constraint(s)
Presolved model has 1 quadratic constraint(s)
Pr